In [1]:
import sys
import json
import numpy as np

from keras.models import Model
# from keras.layers import Input, Embedding, Dropout, Bidirectional, GRU, CuDNNGRU, TimeDistributed, Dense
from keras.layers import Input, Embedding, Dropout, TimeDistributed, Dense, Add, add
from keras.layers import LeakyReLU

from utils import *
from AttentionWithContext import AttentionWithContext
from StructuredSelfAttentive import StructuredSelfAttentive
from AttentionWithMultiContext import AttentionWithMultiContext
from SkipConnection import SkipConnection


from time import time

# = = = = = = = = = = = = = = =

start_time = time()

is_GPU = True

path_root = ''# fill me!
path_to_code = path_root
path_to_data = path_root + 'data/'
sys.path.insert(0, path_to_code)

# = = = = = = = = = = = = = = =

# from AttentionWithContext import AttentionWithContext

# def bidir_gru(my_seq,n_units,is_GPU):
#     '''
#     just a convenient wrapper for bidirectional RNN with GRU units
#     enables CUDA acceleration on GPU
#     # regardless of whether training is done on GPU, model can be loaded on CPU
#     # see: https://github.com/keras-team/keras/pull/9112
#     '''
#     if is_GPU:
#         return Bidirectional(CuDNNGRU(units=n_units,
#                                       return_sequences=True),
#                              merge_mode='concat', weights=None)(my_seq)
#     else:
#         return Bidirectional(GRU(units=n_units,
#                                  activation='tanh', 
#                                  dropout=0.0,
#                                  recurrent_dropout=0.0,
#                                  implementation=1,
#                                  return_sequences=True,
#                                  reset_after=True,
#                                  recurrent_activation='sigmoid'),
#                              merge_mode='concat', weights=None)(my_seq)

# = = = = = = = = = = = = = = =

docs = np.load(path_to_data + 'documents_p2q_5_wl10.npy')
embeddings = np.load(path_to_data + 'embeddings_p2q_5_wl10.npy')

with open(path_to_data + 'train_idxs.txt', 'r') as file:
    train_idxs = file.read().splitlines()

with open(path_to_data + 'test_idxs.txt', 'r') as file:
    test_idxs = file.read().splitlines()

train_idxs = [int(elt) for elt in train_idxs]
test_idxs = [int(elt) for elt in test_idxs]

######
np.random.seed(12219)
idxs_select_train = np.random.choice(range(len(train_idxs)),size=int(len(train_idxs)*0.80),replace=False)
idxs_select_val = np.setdiff1d(range(len(train_idxs)),idxs_select_train)

train_idxs_new = [train_idxs[elt] for elt in idxs_select_train]
val_idxs = [train_idxs[elt] for elt in idxs_select_val]

docs_train = docs[train_idxs_new,:,:] ###
docs_val = docs[val_idxs,:,:] ###
docs_test = docs[test_idxs,:,:]

# = = = = = TRAINING RESULTS = = = = = 

for tgt in range(4):
    
    print('* * * * * * *',tgt,'* * * * * * *')
    
    with open(path_to_data + 'model_history_sc' + str(tgt) + '_sc.json', 'r') as file:
        hist = json.load(file)
    
    val_mse = hist['val_loss']
    val_mae = hist['val_mean_absolute_error']
    
    min_val_mse = min(val_mse)
    min_val_mae = min(val_mae)
    
    best_epoch = val_mse.index(min_val_mse) + 1
    
    print('best epoch:',best_epoch)
    print('best val MSE',round(min_val_mse,3))
    print('best val MAE',round(min_val_mae,3))

# = = = = = PREDICTIONS = = = = =     

# all_preds_mean = []
all_preds_han = []
all_preds_train = [] ###
all_preds_val = [] ###

for tgt in range(4):
    
    print('* * * * * * *',tgt,'* * * * * * *')
    
    # * * * mean baseline * * * 
    
#     with open(path_to_data + 'targets/train/target_' + str(tgt) + '.txt', 'r') as file:
#         target = file.read().splitlines()
    
#     target = np.array(target).astype('float')
#     target_mean = np.mean(target)
#     all_preds_mean.append([target_mean]*len(test_idxs))
    
    # * * * HAN * * * 
    
    # relevant hyper-parameters
    n_units = 60
    ###
    mc_n_units = 100
    da = 15
    r = 10
    ###
    drop_rate = 0 # prediction mode
    
    sent_ints = Input(shape=(docs_test.shape[2],))
    
    sent_wv = Embedding(input_dim=embeddings.shape[0],
                        output_dim=embeddings.shape[1],
                        weights=[embeddings],
                        input_length=docs_test.shape[2],
                        trainable=False,
                        )(sent_ints)
    
#     sent_wv_dr = Dropout(drop_rate)(sent_wv)
#     sent_wa = bidir_gru(sent_wv_dr,n_units,is_GPU)
#     sent_att_vec,word_att_coeffs = AttentionWithContext(return_coefficients=True)(sent_wa)
#     sent_att_vec_dr = Dropout(drop_rate)(sent_att_vec)                      
#     sent_encoder = Model(sent_ints,sent_att_vec_dr)
    
#     doc_ints = Input(shape=(docs_test.shape[1],docs_test.shape[2],))
#     sent_att_vecs_dr = TimeDistributed(sent_encoder)(doc_ints)
#     doc_sa = bidir_gru(sent_att_vecs_dr,n_units,is_GPU)
#     doc_att_vec,sent_att_coeffs = AttentionWithContext(return_coefficients=True)(doc_sa)
#     doc_att_vec_dr = Dropout(drop_rate)(doc_att_vec)
    
#     hid = Dense(units=4,activation='tanh')(doc_att_vec_dr)
#     preds = Dense(units=1)(hid)
    
#     model = Model(doc_ints,preds)

    ## HAN sent encoder
    sent_wv_dr = Dropout(drop_rate)(sent_wv)
    sent_wa = bidir_gru(sent_wv_dr, n_units, is_GPU)
    sent_wa = bidir_gru(sent_wa, n_units, is_GPU) #########
    sent_att_vec, word_att_coeffs = AttentionWithContext(return_coefficients=True)(sent_wa)
    sent_att_vec_dr = Dropout(drop_rate)(sent_att_vec)
    # skip connection
    sent_added = SkipConnection()([sent_att_vec_dr, sent_wv_dr])
    sent_encoder = Model(sent_ints, sent_added)

    ## structured self-attentive
    mc_sent_wv_dr = Dropout(drop_rate)(sent_wv)
    mc_sent_wa = bidir_lstm(mc_sent_wv_dr, mc_n_units, is_GPU)
    mc_sent_wa = bidir_lstm(mc_sent_wa, mc_n_units, is_GPU) #######
    mc_sent_att_vec, mc_word_att_coeffs = StructuredSelfAttentive(da=da, r=r, return_coefficients=True)(mc_sent_wa)
    mc_sent_att_vec_dr = Dropout(drop_rate)(mc_sent_att_vec)
    # skip connection
    mc_sent_added = SkipConnection()([mc_sent_att_vec_dr, mc_sent_wv_dr])
    mc_sent_encoder = Model(sent_ints, mc_sent_added)

    ## combine context and target
    doc_ints = Input(shape=(docs_test.shape[1], docs_test.shape[2],))
    # sentence encoder
    sent_att_vecs_dr = TimeDistributed(sent_encoder)(doc_ints)
    doc_sa = bidir_gru(sent_att_vecs_dr, n_units, is_GPU)
    # context
    mc_sent_att_vecs_dr = TimeDistributed(mc_sent_encoder)(doc_ints)
    mc_doc_sa = bidir_gru(mc_sent_att_vecs_dr, n_units, is_GPU)

    doc_att_vec, sent_att_coeffs = AttentionWithMultiContext(return_coefficients=True)([doc_sa, mc_doc_sa])
    doc_att_vec_dr = Dropout(drop_rate)(doc_att_vec)

    # new
    # hid = Dense(units=4, activation='tanh')(doc_att_vec_dr)
    hid1 = LeakyReLU(alpha=0.01)(doc_att_vec_dr)
    hid2 = LeakyReLU(alpha=0.01)(hid1)
    preds = Dense(units=1)(hid2)

    model = Model(doc_ints, preds)
    
    model.load_weights(path_to_data + 'model_sc' + str(tgt))
    
    all_preds_han.append(model.predict(docs_test).tolist())
    all_preds_train.append(model.predict(docs_train).tolist()) ###
    all_preds_val.append(model.predict(docs_val).tolist()) ###

# flatten
# all_preds_mean = [elt for sublist in all_preds_mean for elt in sublist]
all_preds_han = [elt[0] for sublist in all_preds_han for elt in sublist]
all_preds_train = [elt[0] for sublist in all_preds_train for elt in sublist]
all_preds_val = [elt[0] for sublist in all_preds_val for elt in sublist]

# write predictions in Kaggle format
# with open(path_to_data + 'pred_mean.txt', 'w') as file:
#     file.write('id,pred\n')
#     for idx,pred in enumerate(all_preds_mean):
#         pred = format(pred, '.7f')
#         file.write(str(idx) + ',' + pred + '\n')

with open(path_to_data + 'pred_han_sc.txt', 'w') as file:
    file.write('id,pred\n')
    for idx,pred in enumerate(all_preds_han):
        pred = format(pred, '.7f')
        file.write(str(idx) + ',' + pred + '\n')

with open(path_to_data + 'pred_train_sc.txt', 'w') as file:
    file.write('id,pred\n')
    for idx,pred in enumerate(all_preds_train):
        pred = format(pred, '.7f')
        file.write(str(idx) + ',' + pred + '\n')

with open(path_to_data + 'pred_val_sc.txt', 'w') as file:
    file.write('id,pred\n')
    for idx,pred in enumerate(all_preds_val):
        pred = format(pred, '.7f')
        file.write(str(idx) + ',' + pred + '\n')

print('everything done in', round(time() - start_time,1))


Using TensorFlow backend.


* * * * * * * 0 * * * * * * *
best epoch: 33
best val MSE 0.161
best val MAE 0.297
* * * * * * * 1 * * * * * * *
best epoch: 36
best val MSE 0.073
best val MAE 0.191
* * * * * * * 2 * * * * * * *
best epoch: 24
best val MSE 0.379
best val MAE 0.424
* * * * * * * 3 * * * * * * *
best epoch: 33
best val MSE 0.008
best val MAE 0.058
* * * * * * * 0 * * * * * * *
Instructions for updating:
Colocations handled automatically by placer.
* * * * * * * 1 * * * * * * *
* * * * * * * 2 * * * * * * *
* * * * * * * 3 * * * * * * *
everything done in 488.6
